In [1]:
from deeppavlov import train_model, build_model 
from deeppavlov.core.commands.utils import parse_config

PROJECT_DIR = '..'

model_config = parse_config('ner_collection3_bert')

# dataset that the model was trained on
print(model_config['dataset_reader']['data_path'])

model_config['dataset_reader']['data_path'] = PROJECT_DIR + '/datasets/conll/'

del model_config['metadata']['download']

model_config['metadata']['variables']['MODEL_PATH'] = PROJECT_DIR + '/models'

model_config['chainer']['pipe'][1]['save_path'] = PROJECT_DIR + '/models/tag.dict'
model_config['chainer']['pipe'][1]['load_path'] = PROJECT_DIR + '/models/tag.dict'

model_config['chainer']['pipe'][2]['save_path'] = PROJECT_DIR + '/models'
model_config['chainer']['pipe'][2]['load_path'] = PROJECT_DIR + '/models'

ner_model = build_model(model_config, download=False)

~/.deeppavlov/downloads/collection3/


c:\Users\lexan\miniconda3\envs\dp310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\lexan\miniconda3\envs\dp310\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm

In [11]:
ner_model(['стаканы рюм'])

[[['стаканы', 'рюм']], [['B-TYPE', 'E-TYPE']]]

In [2]:
import re
from pathlib import Path

import pandas as pd

SUBMISSION_PATH = Path(PROJECT_DIR) / 'datasets' / 'submission_raw.csv'
OUTPUT_PATH = Path(PROJECT_DIR) / 'datasets' / 'submission.csv'

TOKEN_PATTERN = re.compile(r'\S+')

def normalize_tag(tag: str) -> str:
    if not isinstance(tag, str):
        return 'O'
    tag = tag.strip()
    if not tag:
        return 'O'
    upper_tag = tag.upper()
    if upper_tag == 'O':
        return 'O'
    prefix = upper_tag[:2]
    if prefix == 'S-':
        return 'B-' + tag[2:]
    if prefix == 'E-':
        return 'I-' + tag[2:]
    if prefix in ('B-', 'I-'):
        return prefix + tag[2:]
    return tag

def is_tag(value) -> bool:
    if not isinstance(value, str):
        return False
    candidate = value.strip().upper()
    if not candidate:
        return False
    if candidate == 'O':
        return True
    return len(candidate) >= 3 and candidate[1] == '-' and candidate[0] in {'B', 'I', 'S', 'E'}

def looks_like_sequence(seq, predicate) -> bool:
    if not isinstance(seq, (list, tuple)) or not seq:
        return False
    return all(predicate(item) for item in seq)

def looks_like_tag_sequence(seq) -> bool:
    return looks_like_sequence(seq, is_tag)

def looks_like_token_sequence(seq) -> bool:
    return looks_like_sequence(seq, lambda item: isinstance(item, str) and not is_tag(item))

def extract_tokens_and_tags(prediction) -> tuple[list[str], list[str]]:
    if isinstance(prediction, tuple):
        prediction = list(prediction)
    if not isinstance(prediction, list):
        raise ValueError(f'Unexpected model output type: {type(prediction)}')
    tokens: list[str] = []
    tags: list[str] = []

    def traverse(node):
        nonlocal tokens, tags
        if isinstance(node, tuple):
            node = list(node)
        if isinstance(node, list):
            if not tokens and looks_like_token_sequence(node):
                tokens = [str(item) for item in node]
            if not tags and looks_like_tag_sequence(node):
                tags = [normalize_tag(str(item)) for item in node]
            if tokens and tags:
                return
            for child in node:
                traverse(child)

    traverse(prediction)
    if not tags:
        raise ValueError(f'Unable to extract tag sequence from model output: {prediction}')
    return tokens, tags

def compute_annotation(text: str, tokens: list[str], tags: list[str]) -> list[tuple[int, int, str]]:
    if not tags:
        return []
    if tokens:
        effective_len = min(len(tokens), len(tags))
        tokens = tokens[:effective_len]
        tags = tags[:effective_len]
    annotation: list[tuple[int, int, str]] = []
    if tokens:
        cursor = 0
        fallback = False
        for token, tag in zip(tokens, tags):
            token = token or ''
            if not tag:
                cursor += len(token)
                continue
            start = text.find(token, cursor)
            if start == -1:
                fallback = True
                break
            end = start + len(token)
            annotation.append((start, end, tag))
            cursor = end
        if fallback:
            annotation = []
    if not annotation:
        matches = list(TOKEN_PATTERN.finditer(text))
        effective_len = min(len(matches), len(tags))
        for match, tag in zip(matches[:effective_len], tags[:effective_len]):
            if not tag:
                continue
            annotation.append((match.start(), match.end(), tag))
    return annotation

submission_df = pd.read_csv(SUBMISSION_PATH, sep=';', encoding='utf-8')
annotations = []
for sample in submission_df['sample']:
    text = '' if pd.isna(sample) else str(sample)
    model_output = ner_model([text])
    tokens, tags = extract_tokens_and_tags(model_output)
    annotations.append(compute_annotation(text, tokens, tags))

submission_df['annotation'] = [str(ann) for ann in annotations]
submission_df[['sample', 'annotation']].to_csv(OUTPUT_PATH, sep=';', encoding='utf-8', index=False)
print(f'Saved predictions to {OUTPUT_PATH.resolve()}')



Saved predictions to C:\Users\lexan\OneDrive\Documents\hackaton_lct\datasets\submission.csv


In [7]:
import pandas as pd

submission = pd.read_csv('../datasets/submission.csv', sep=';')


In [ ]:
'''
Напиши код, который пройдется по всем столбца 'sample' в submission, применит ner_model к каждому из них.
На выходе из ner_model получается список строк с типами сущностей. Нужно преобразовать это в новый формат.
Во-первых, нужно заменить все S- на B-, а все E- на I-.
Во-вторых, нужно сделать аннотацию формата [(индекс начала сущности, индекс конца сущности, строка типа),] 
Например, для строки "йогурты питьевы" аннотация будет [(0, 7, 'B-TYPE'), (8, 15, 'I-TYPE')], индексы начала и конца сущности работают как срезы в питоне - включительно-исключительно.
То есть, 0 индекс это буква "й", 7 индекс - это пробел после слова "йогурты", но пробел в саму сущность не входит.
В новый файл submission_final.csv нужно в том же порядке записывать столбцы sample и annotation (аннотации уже в новом формате)
'''

"\nНапиши код, который пройдется по всем столбца 'sample' в submission, применит ner_model к каждому из них.\nНа выходе из ner_model получается список строк с типами сущностей. Нужно преобразовать это в новый формат.\nВо-первых, нужно заменить все S- на B-, а все E- на I-.\nВо-вторых, нужно сделать аннотацию формата [(индекс начала сущности, индекс конца сущности, строка типа),] йогурты питьевы\t[(0, 7, 'B-TYPE'), (8, 15, 'I-TYPE')]\n"